<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [38]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [39]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-13 09:19:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-13 09:19:58 (69.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [40]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-13 09:19:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-09-13 09:19:58 (114 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [41]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [42]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [43]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [44]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [45]:
len(train_df),len(test_df)

(7613, 3263)

In [46]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 Classic WWW ÛÓ Hiroshima/Nagasaki: Atomic Devastation Hidden for Decades: The people of Hiroshima and Nagasaki ... http://t.co/pLo2QkrWHu 

---

target:0 (not a disaster)
Text
 and he almost collapsed bc he said his wish came true moderately FUCK 

---

target:0 (not a disaster)
Text
 @bonhomme37 wouldn't that have sunk the sub? 

---

target:1 (real disaster)
Text
 ISIL claims suicide bombing at Saudi mosque that killed at least 15 http://t.co/Y8IcF89H6w http://t.co/t9MSnZV1Kb 

---

target:1 (real disaster)
Text
 #Cyprus: News Analysis: Mullah Omar's death may split Taliban's ranks - ..Omar's demise would certainly lead... http://t.co/AJkmcusWHo 

---



## splitting the data into training and validation sets

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [49]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [50]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [51]:
max_vocab_length = 10000
max_length = 15

In [52]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [53]:
text_vectorizer.adapt(train_sentences)

In [54]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [55]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 Sitting in a cafe enjoying a bite and cramming for my meeting during my whirlwind 14-hours in NYC! https://t.co/TO0BPiEymS       
 Vectorized version:
 [[1474    4    3 6040 3027    3 6096    7    1   10   13 2051  246   13
   411]]


In [56]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [57]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [58]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Currently want to drive my car off a cliff and fall to my death.         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01896416, -0.01237669, -0.02078601, ...,  0.04347589,
          0.04501469, -0.03385407],
        [ 0.00031558, -0.01730562,  0.01734051, ...,  0.03354465,
          0.04622226, -0.04608394],
        [ 0.04541718,  0.00981114,  0.04712292, ..., -0.01742123,
         -0.00487993, -0.02489228],
        ...,
        [-0.0211797 ,  0.03256488, -0.04341632, ...,  0.03562791,
         -0.04836221,  0.00032924],
        [-0.04228163,  0.01426337, -0.03253545, ..., -0.0198275 ,
         -0.01551306,  0.02832163],
        [ 0.04661993, -0.04449379,  0.01715929, ...,  0.02534703,
         -0.01201477,  0.03620179]]], dtype=float32)>

In [59]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('Currently want to drive my car off a cliff and fall to my death.',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01896416, -0.01237669, -0.02078601, -0.00238287, -0.04728977,
        -0.00601484, -0.00683462, -0.02492907, -0.01446432,  0.00584344,
        -0.03874676, -0.02074751, -0.03294526,  0.0465886 ,  0.04829598,
        -0.00158064, -0.03148057,  0.04291905, -0.0193541 , -0.03564184,
         0.02390505,  0.04029865,  0.03205626,  0.00867603,  0.02535674,
         0.00946896,  0.01527881,  0.01447341, -0.04263053, -0.04833166,
         0.02770949, -0.00197161, -0.04345629,  0.04096944,  0.01688704,
        -0.03162904,  0.04526928,  0.04756606,  0.03822759, -0.03928902,
        -0.04520402,  0.01833152, -0.0062109 , -0.04519305, -0.03234076,
        -0.01441816, -0.01172373,  0.03277263, -0.03930583, -0.030575  ,
        -0.0430931 , -0.01494048,  0.00133352,  0.04589893,  0.00057942,
         0.04297305,  0.02735404, -0.03340179, -0.03075278,  0.02893298,
      

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [61]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [62]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [63]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [64]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [65]:
from helper_functions import create_tensorboard_callback

In [66]:
SAVE_DIR = 'model_logs'

In [67]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [68]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220913-091914
Epoch 1/5
215/215 [==============================] - 8s 31ms/step - loss: 0.6108 - accuracy: 0.6946 - val_loss: 0.5345 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.4404 - accuracy: 0.8164 - val_loss: 0.4701 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3454 - accuracy: 0.8605 - val_loss: 0.4581 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2826 - accuracy: 0.8923 - val_loss: 0.4687 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2354 - accuracy: 0.9142 - val_loss: 0.4806 - val_accuracy: 0.7835


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4806 - accuracy: 0.7835


[0.48060470819473267, 0.7834645509719849]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.3504342 ],
       [0.7171489 ],
       [0.9974338 ],
       [0.13660663],
       [0.11382881],
       [0.9380516 ],
       [0.8998508 ],
       [0.99034667],
       [0.9630409 ],
       [0.3172127 ]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7834645669291339,
 'precision': 0.7880043492637875,
 'recall': 0.7834645669291339,
 'f1': 0.7804929895574078}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])

## Visualising learned embeddings

In [69]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [70]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [75]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights.shape

(10000, 128)

In [76]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [77]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>